# Summary
In this notebook I gather BGC classes information from mibig jsons. This continues from notebook 2 where I gathered CF and NPC chemical class information.

In [1]:
import os
import urllib
import glob
import json
from rdkit import Chem

In [2]:
mibig_folder = "/mnt/scratch/louwe015/mibig_json_2.0/"

base_path = "/mnt/scratch/louwe015/NPLinker/classifying/mibig_classifications/"
mibig_smiles_path = os.path.join(base_path, "files/All_MIBiG_compounds_with_SMILES_and_PMID_MAS.txt")
print("mibig_smiles_path exists is", os.path.isfile(mibig_smiles_path))

mibig_smiles_path exists is True


In [6]:
chem_classes_mibig = os.path.join(os.path.split(base_path[:-1])[0], "All_MIBiG_compounds_with_CF_NPC_classes.txt")
print(chem_classes_mibig, os.path.isfile(chem_classes_mibig))

/mnt/scratch/louwe015/NPLinker/classifying/All_MIBiG_compounds_with_CF_NPC_classes.txt True


## Which entries are needed from the jsons?
Original list from Oscar:

['compound_name','biosyn_class','biosyn_subclass',\
    'chem_synonyms','chem_target','molecular_formula','mol_mass',\
    'chem_struct','pubchem_id','chemspider_id','chebi_id','chembl_id',\
    'chem_act','other_chem_act','loci','publications','rdkit_smile_1',\
    'rdkit_smile_2','rdkit_smile_3','rdkit_smile_4','rdkit_inchi_key',\
    'rdkit_inchi_key1']

In [7]:
# as ids take BGC id + '_' + compound name
wanted_names = ['compound_name','biosyn_class','biosyn_subclass',\
    'chem_synonyms','chem_target','molecular_formula','mol_mass',\
    'chem_struct','pubchem_id','chemspider_id','chebi_id','chembl_id',\
    'chem_act','other_chem_act','loci','publications']

In [8]:
mibig_files = glob.glob(os.path.join(mibig_folder, "*.json"))

In [25]:
test = mibig_files[4]
test

'/mnt/scratch/louwe015/mibig_json_2.0/BGC0001001.json'

In [26]:
with open(test) as inf:
    test_json = json.load(inf)

In [32]:
test_json["cluster"]

{'biosyn_class': ['NRP', 'Polyketide'],
 'compounds': [{'chem_struct': 'CO\\C(CCNC(=O)CC\\C=C\\C(C)CC\\C(CCCC#CBr)=C\\Cl)=C\\C(=O)N1C(C)C=CC1=O',
   'compound': 'jamaicamide A',
   'database_id': ['npatlas:13036'],
   'mol_mass': 566.1546974120001,
   'molecular_formula': 'C27H36BrClN2O4'},
  {'chem_acts': ['Sodium channel blocking'],
   'chem_struct': 'CO\\C(CCNC(=O)CC\\C=C\\C(C)CC\\C(CCCC#C)=C\\Cl)=C\\C(=O)N1C(C)C=CC1=O',
   'chem_targets': [{'target': 'Sodium channel blocking'}],
   'compound': 'jamaicamide B',
   'database_id': ['npatlas:374', 'pubchem:49787032'],
   'mol_mass': 488.244185344,
   'molecular_formula': 'C27H37ClN2O4'},
  {'chem_struct': 'CO\\C(CCNC(=O)CC\\C=C\\C(C)CC\\C(CCCC=C)=C\\Cl)=C\\C(=O)N1C(C)C=CC1=O',
   'chem_targets': [{'target': 'Sodium channel blocking'}],
   'compound': 'jamaicamide C',
   'database_id': ['npatlas:1111', 'pubchem:49787033'],
   'mol_mass': 490.259835408,
   'molecular_formula': 'C27H39ClN2O4'}],
 'genes': {'annotations': [{'id': 'AAS98794

## Investigate subclasses

In [44]:
all_class_info = []
for mibig_file in mibig_files:
    with open(mibig_file) as inf:
        mibig_js = json.load(inf)
    class_list = mibig_js["cluster"].get("biosyn_class")
    if class_list:
        for elem in class_list:
            low_elem = elem.lower()
            class_dict = mibig_js["cluster"].get(low_elem)
            if class_dict:
                all_class_info.append(class_dict)

In [48]:
print(len(all_class_info))
print(len([dct["subclass"] for dct in all_class_info if "subclass" in dct]))
print(len([dct["subclasses"] for dct in all_class_info if "subclasses" in dct]))

759
363
236
